In [3]:
import pandas as pd
from langchain_core.documents import Document

In [4]:
def dataprocessing():
    product_data=pd.read_csv("/Users/rasikagulhane/Desktop/Customer_Rating_Reviews/data/Raw_Reviews.csv")

    data=product_data[["Type","Text_Review"]]

    product_list = []

    # Iterate over the rows of the DataFrame
    for index, row in data.iterrows():
        # Construct an object with 'product_name' and 'review' attributes
        obj = {
                'product_category': row['Type'],
                'review': row['Text_Review']
            }
        # Append the object to the list
        product_list.append(obj)
        

    docs = []
    for entry in product_list:
        metadata = {"product_category": entry['product_category']}
        doc = Document(page_content=entry['review'], metadata=metadata)
        docs.append(doc)
    return docs

In [5]:
dataprocessing()[:10]

[Document(page_content='Absolutely wonderful - silky and sexy and comfortable', metadata={'product_category': 'Intimate'}),
 Document(page_content='Love this dress!  it\'s sooo pretty.  i happened to find it in a store, and i\'m glad i did bc i never would have ordered it online bc it\'s petite.  i bought a petite and am 5\'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.', metadata={'product_category': 'Dresses'}),
 Document(page_content='I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn direct

In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY=os.getenv("open_ai_key")

In [7]:
from dotenv import load_dotenv
import os

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
ASTRA_DB_API_ENDPOINT=os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN=os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE=os.getenv("ASTRA_DB_KEYSPACE")

### vector database configuration

In [8]:
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings

import os
import pandas as pd
from retailbot.datapreprocessing import dataprocessing

from dotenv import load_dotenv
load_dotenv()

True

In [9]:
import os
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

In [10]:

# from constants.sec import open_ai_api_key
# os.environ['OPENAI_API_KEY']= open_ai_api_key


OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
ASTRA_DB_API_ENDPOINT=os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN=os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE=os.getenv("ASTRA_DB_KEYSPACE")

In [11]:

embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY)


In [12]:
def ingestdata(status):
    vstore = AstraDBVectorStore(
            embedding=embedding,
            collection_name="retail_reviews",
            api_endpoint=ASTRA_DB_API_ENDPOINT,
            token=ASTRA_DB_APPLICATION_TOKEN,
            namespace=ASTRA_DB_KEYSPACE,
        )
    
    storage=status
    
    if storage==None:
        docs=dataprocessing()
        batch_size = 10  # Adjust this based on your needs and rate limit
        inserted_ids = []
        for i in range(0, len(docs), batch_size):
            batch = docs[i:i+batch_size]
            inserted_ids.extend(vstore.add_documents(batch))
    else:
        return vstore
    return vstore, inserted_ids

In [13]:


if __name__=='__main__':
    vstore,inserted_ids=ingestdata(None)
    # print(f"\nInserted {len(inserted_ids)} documents.")
    # results = vstore.similarity_search("can you tell me the low budget sound basshead.")
    # for res in results:
    #         print(f"* {res.page_content} [{res.metadata}]")
            

   

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-embedding-ada-002 in organization org-s8euPC4UL1hWFQgKrADrBn1k on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}